In [2]:
import torch

In [4]:
x = torch.ones(2,2,dtype=int)
x.dtype

torch.int64

In [7]:
x = torch.rand(2,2)
y = torch.rand(2,2)
z = x/y
x,y,z

(tensor([[0.6838, 0.1484],
         [0.3328, 0.0787]]),
 tensor([[0.9779, 0.9992],
         [0.9861, 0.2324]]),
 tensor([[0.6992, 0.1485],
         [0.3375, 0.3387]]))

In [8]:
x[1,1], x[1,1].item()

(tensor(0.0787), 0.07870721817016602)

In [10]:
x = torch.rand(4,4)
print(x)
print(x.view(16))
print(x.view(-1,8))
print(x.view(2,-1))

tensor([[0.5116, 0.7009, 0.8786, 0.2163],
        [0.6351, 0.6476, 0.9121, 0.2947],
        [0.0952, 0.4026, 0.3520, 0.8690],
        [0.3217, 0.8022, 0.5578, 0.5156]])
tensor([0.5116, 0.7009, 0.8786, 0.2163, 0.6351, 0.6476, 0.9121, 0.2947, 0.0952,
        0.4026, 0.3520, 0.8690, 0.3217, 0.8022, 0.5578, 0.5156])
tensor([[0.5116, 0.7009, 0.8786, 0.2163, 0.6351, 0.6476, 0.9121, 0.2947],
        [0.0952, 0.4026, 0.3520, 0.8690, 0.3217, 0.8022, 0.5578, 0.5156]])
tensor([[0.5116, 0.7009, 0.8786, 0.2163, 0.6351, 0.6476, 0.9121, 0.2947],
        [0.0952, 0.4026, 0.3520, 0.8690, 0.3217, 0.8022, 0.5578, 0.5156]])


In [11]:
print(torch.cuda.is_available())

False


# Autograd

In [20]:
x = torch.randn(3, requires_grad=True)
y = x + 2
z = y*y*2
z = z.mean()
print(z)
z.backward()
x.grad ### that's how you access gradients

tensor(10.7862, grad_fn=<MeanBackward0>)


tensor([1.4404, 1.5244, 4.9361])

In [17]:
x = torch.randn(3, requires_grad=True)
print(x)
#x.requires_grad_(False)
#x.detach()
#with torch.no_grad():

y = x.detach()

tensor([0.5235, 1.3156, 0.2277], requires_grad=True)


In [19]:
x = x + 2
print(y, x)

tensor([0.5235, 1.3156, 0.2277]) tensor([2.5235, 3.3156, 2.2277], grad_fn=<AddBackward0>)


In [22]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    model_output = (weights*3).sum()

    model_output.backward()

    print(weights.grad)

tensor([3., 3., 3., 3.])
tensor([6., 6., 6., 6.])
tensor([9., 9., 9., 9.])


In [23]:
weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
    model_output = (weights*3).sum()

    model_output.backward()

    print(weights.grad)
    weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


In [26]:
x = torch.tensor(1.0)
y = torch.tensor(2.0)

w = torch.tensor(1.0, requires_grad=True)

### forward pass
y_hat = w * x
loss = (y_hat - y)**2 

print(loss)

### backward
loss.backward()
print(w.grad)


tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)


# Training pipe line

In [5]:
import torch.nn as nn
X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

def forward(x):
    return w * x

loss = nn.MSELoss()
optimizer = torch.optim.SGD([w], lr=0.01) 

for epoch in range(100):

    y_pred = forward(X)
    
    l = loss(Y, y_pred)

    l.backward()

    optimizer.step()
    
    optimizer.zero_grad()

print(forward(5))

tensor(10.0000, grad_fn=<MulBackward0>)


In [11]:
import torch.nn as nn
X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32) 
n_samples, n_features = X.shape
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

model = nn.Linear(n_features, n_features)

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

for epoch in range(10000):

    y_pred = model(X)
    
    l = loss(Y, y_pred)

    l.backward()

    optimizer.step()
    
    optimizer.zero_grad()

print(model(X_test))

tensor([10.0000], grad_fn=<AddBackward0>)


In [13]:
import torch.nn as nn
X = torch.tensor([[1], [2], [3], [4]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32) 
n_samples, n_features = X.shape
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

##model = nn.Linear(n_features, n_features)

class LinearRegression(nn.Module):

    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)

model = LinearRegression(n_features, n_features)
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

for epoch in range(10000):

    y_pred = model(X)
    
    l = loss(Y, y_pred)

    l.backward()

    optimizer.step()
    
    optimizer.zero_grad()

print(model(X_test))

tensor([10.0000], grad_fn=<AddBackward0>)


In [ ]:
'''
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet
complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html
On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale
On Tensors
----------
LinearTransformation, Normalize, RandomErasing
Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage
Generic
-------
Use Lambda 
Custom
------
Write own class
Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''

import torch
import torchvision
from torch.utils.data import Dataset
import numpy as np


class WineDataset(Dataset):

    def __init__(self, transform=None):
        xy = np.loadtxt('./data/wine/wine.csv', delimiter=',', dtype=np.float32, skiprows=1)
        self.n_samples = xy.shape[0]

        # note that we do not convert to tensor here
        self.x_data = xy[:, 1:]
        self.y_data = xy[:, [0]]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

    def __len__(self):
        return self.n_samples

# Custom Transforms
# implement __call__(self, sample)
class ToTensor:
    # Convert ndarrays to Tensors
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

class MulTransform:
    # multiply inputs with a given factor
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

print('Without Transform')
dataset = WineDataset()
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

print('\nWith Tensor Transform')
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

print('\nWith Tensor and Multiplication Transform')
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

In [ ]:
!pip3 install torchvision

In [19]:
import torchvision

dataset = torchvision.datasets.MNIST(root='./data', transform=torchvision.transforms.ToTensor(), download=True)

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.linear(x)
        out = self.relu(out)
        out = self.linear(out)
        out = self.sigmoid(out)
        return out

In [20]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [21]:
input_size = 784 
hidden_size = 100
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

In [24]:
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transforms.ToTensor(), download=False)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = True) 

In [26]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1,28*28)

        output = model(images)
        print(output.shape)
        loss = criterion(output, labels)

        optimizer.zero_grad()   
        loss.backward()

In [29]:
a = torch.tensor([1,2,-5,0.2,3])
torch.min(torch.ones(a.size()), a)

tensor([ 1.0000,  1.0000, -5.0000,  0.2000,  1.0000])